In [1]:
from omegaconf import OmegaConf
import torch
config_file = "./custom_configs/7B_lora.yaml"

In [50]:
cfg = OmegaConf.load(config_file)

In [51]:
from torchtune.config._instantiate import instantiate

In [53]:
scheduler_cfg = cfg.profile.schedule
scheduler_cfg.repeat = 0

In [54]:
test_schedule = instantiate(scheduler_cfg)

In [55]:
ref_schedule = torch.profiler.schedule(wait=1, warmup=1, active=1)

In [56]:
test_actions = [test_schedule(i) for i in range(5)]
ref_actions = [ref_schedule(i) for i in range(5)]
assert (test_actions == ref_actions)

In [85]:
cfg = OmegaConf.load(config_file)
cfg.profile.profiler

{'_component_': 'torch.profiler.profile', 'record_shapes': True, 'profile_memory': True, 'with_stack': True, 'with_flops': True}

In [89]:
cfg.profile.CUDA
OmegaConf.select(cfg.profile, "CPU", default=True, throw_on_missing=False)

True

In [77]:

p = instantiate(cfg.profile.profiler, activities=[torch.profiler.ProfilerActivity.CPU if cfg.profile.CPU else None, torch.profiler.ProfilerActivity.CUDA if cfg.profile.CUDA else None])

In [80]:
type(p.activities)

set

In [81]:
assert all(a in p.activities for a in [torch.profiler.ProfilerActivity.CPU, torch.profiler.ProfilerActivity.CUDA])

In [82]:
ref_prof = torch.profiler.profile(activities=[torch.profiler.ProfilerActivity.CPU, torch.profiler.ProfilerActivity.CUDA], record_shapes=True, profile_memory=True, with_stack=True, with_flops=True)

In [84]:
assert p.activities == ref_prof.activities